In [1]:
import os
import gc
import json
from functools import reduce
import pandas as pd

import dask.dataframe as dd
import dask.bag as db  # 리스트와 유사하며 큰 데이터셋을 처리
from dask.distributed import Client  # DASK의 분산 스케줄러 설정

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Attend 데이터 df_attend로 로드

In [2]:
attend_directory = "D:/DATA_PREPROCESS/FIRESTORE_DATAS/USERS"
# attend_file_names = [f for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]

# Spark 세션 생성 및 전역 변수로 설정
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.maxResultSize", "3g") \
    .getOrCreate()

In [3]:
def load_json_to_df(directory):
    """해당 디렉토리로부터 'output-'형식의 JSON 파일들을 DataFrame으로 로드"""
    file_names = [f for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]
#     file_names = [f"output-{i}.json" for i in range(11)]
    
    dfs_chunk = []  # 이 리스트는 한 파일의 모든 청크 데이터프레임을 저장합니다.

    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            
            users_data = data.get("Users", {})
            
            for user_id, user_info in users_data.items():
                attend_data = user_info.get("Attend", {})
                for attend_name, attend_info in attend_data.items():
                    attend_df = pd.DataFrame([attend_info])  # Convert each attend_info into a dataframe
                    attend_df['user_id'] = user_id  # Adding user ID
                    dfs_chunk.append(attend_df)
                    
            # Log to check the processed files
            print(f"Processed file: {file_name}")

        except Exception as e:
            print(f"Error reading file: {file_name}. Reason: {str(e)}. Skipping this file.")

    # Concatenating all dataframes from all files
    if dfs_chunk:
        final_df = pd.concat(dfs_chunk, ignore_index=True)
        return final_df
    else:
        print("No dataframes to concatenate.")
        return None

In [4]:
# def chunk_files(file_list, size):
#     """주어진 파일 리스트를 그룹화합니다."""
#     return [file_list[i:i + size] for i in range(0, len(file_list), size)]

def save_progress(part_number):
    """작업의 진행 상황을 저장합니다."""
    directory = 'D:/DATA_PREPROCESS/FIRESTORE_DATAS/data_progress'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(os.path.join(directory, 'progress.txt'), 'w') as file:
        file.write(str(part_number))

def load_progress():
    """작업의 진행 상황을 불러옵니다."""
    if os.path.exists('D:/DATA_PREPROCESS/FIRESTORE_DATAS/data_progress/progress.txt'):
        with open('D:/DATA_PREPROCESS/FIRESTORE_DATAS/data_progress/progress.txt', 'r') as file:
            content = file.read().strip()
            if content.isdigit():  # 파일의 내용이 숫자인지 확인
                return int(content)
    return 0  # 진행 상황 파일이 없으면 처음부터 시작

# def process_json_file(file_path):
#     # JSON 파일을 Spark DataFrame으로 읽기
#     df = spark.read.option("multiLine", "true").json(file_path)
    
#     # Users 딕셔너리를 배열로 변환하고 explode 함수 적용
#     exploded_df = df.select(F.explode(F.col("Users"))).toDF("user_id", "user_info")
    
#     # Attend 정보를 추출합니다.
#     attend_df = exploded_df.select(
#         "user_id", 
#         F.explode("user_info.Attend").alias("attend_name", "attend_info"),
#         F.col("attend_info.ArrayVoice").alias("ArrayVoice"),
#         F.col("attend_info.attend").alias("attend"),
#         F.col("attend_info.resultPass").alias("resultPass"),
#         F.col("attend_info.imageUrl").alias("imageUrl"),
#         F.col("attend_info.addText").alias("addText"),
#         F.col("attend_info.name").alias("name"),
#         F.col("attend_info.ArrayPercent").alias("ArrayPercent"),
#         F.col("attend_info.ArrayDate").alias("ArrayDate"),
#         F.col("attend_info.title").alias("title"),
#         F.col("attend_info.type").alias("type"),
#         F.col("attend_info.review_wish").alias("review_wish"),
#         F.col("attend_info.ArrayAddResult").alias("ArrayAddResult")
#     )
    
#     return attend_df
def process_json_file_spark(file_path):
    """주어진 JSON 파일 경로에서 Users 및 Attend 정보를 처리하여 Spark DataFrame으로 반환합니다."""
    # JSON 파일을 Spark DataFrame으로 읽기
    df = spark.read.option("multiLine", "true").json(file_path)
    
    # Users 딕셔너리를 배열로 변환하고 explode 함수 적용
    exploded_users_df = df.select(F.explode(F.col("Users"))).toDF("user_id", "user_info")
    
    # Attend 정보 추출
    attend_df = exploded_users_df.select(
        "user_id", 
        F.explode("user_info.Attend").alias("attend_name", "attend_info")
    )
    
    # 필요한 컬럼 추출
    final_df = attend_df.select(
        "user_id",
        "attend_name",
        F.col("attend_info.ArrayVoice").alias("ArrayVoice"),
        F.col("attend_info.attend").alias("attend"),
        F.col("attend_info.resultPass").alias("resultPass"),
        F.col("attend_info.imageUrl").alias("imageUrl"),
        F.col("attend_info.addText").alias("addText"),
        F.col("attend_info.name").alias("name"),
        F.col("attend_info.ArrayPercent").alias("ArrayPercent"),
        F.col("attend_info.ArrayDate").alias("ArrayDate"),
        F.col("attend_info.title").alias("title"),
        F.col("attend_info.type").alias("type"),
        F.col("attend_info.review_wish").alias("review_wish"),
        F.col("attend_info.ArrayAddResult").alias("ArrayAddResult")
    )
    
    return final_df

In [5]:
# def load_json_to_df_spark(directory: str) -> pd.DataFrame:
#     file_names = [f for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]
#     final_dfs = []
    
#     last_completed_part = load_progress()

#     for idx, file_name in enumerate(file_names):
#         file_path = os.path.join(directory, file_name)
#         current_df = process_json_file(file_path)
        
#         temp_dfs = []
#         user_ids = current_df.select("user_id").distinct().rdd.flatMap(lambda x: x).collect()
        
#         for i in range(0, len(user_ids), 100):
#             current_users = user_ids[i:i+100]
#             chunk_df = current_df.filter(current_df.user_id.isin(current_users))
#             temp_dfs.append(chunk_df)
            
#             # 중간 결과 저장
#             if (idx * len(user_ids) + i) > last_completed_part:
#                 final_dfs.append(chunk_df)
#                 save_progress(idx * len(user_ids) + i)
                
#     final_df = reduce(DataFrame.unionByName, final_dfs)
#     return final_df.toPandas()
def load_json_to_df_spark(directory):
    """해당 디렉토리로부터 'output-'형식의 JSON 파일들을 DataFrame으로 로드"""
    file_names = [f for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]
    
    # 진행 상황 로딩
    last_completed_file_index = load_progress()
    
    # 모든 파일에 대해 처리 후 결과 DataFrame들을 병합
    all_dfs = []
    for idx, file_name in enumerate(file_names):
        # 이전에 완료된 파일은 스킵
        if idx < last_completed_file_index:
            continue
            
        file_path = os.path.join(directory, file_name)
        current_df = process_json_file_spark(file_path)
        all_dfs.append(current_df)
        
        # 현재 파일 인덱스를 진행 상황으로 저장
        save_progress(idx + 1)

    final_df = reduce(lambda x, y: x.union(y), all_dfs)
    return final_df

In [6]:
def load_json_to_df_with_dask(directory):
    # DASK 작업이 분산 스케줄러에 의해 처리되게 함
    client = Client(memory_limit='4GB')  # 시스템 메모리의 4GB를 사용하도록 설정
    
    file_names = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]
#     file_names = [os.path.join(directory, f) for f in os.listdir(directory) 
#               if f.startswith('output-') and f.endswith('.json') 
#               and 0 <= int(f.split('-')[1].split('.')[0]) <= 10]
    
    # JSON 파일 로드 및 전처리를 위한 함수
    def process_file(file_path):
        dfs_chunk = []
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            users_data = data.get("Users", {})

            for user_id, user_info in users_data.items():
                attend_data = user_info.get("Attend", {})
                for attend_name, attend_info in attend_data.items():
                    attend_df = dd.from_pandas(pd.DataFrame([attend_info]), npartitions=1)  # Convert each attend_info into a dask dataframe
                    attend_df['user_id'] = user_id
                    dfs_chunk.append(attend_df)
        except Exception as e:
            print(f"Error reading file: {file_name}. Reason: {str(e)}. Skipping this file.")
        return dfs_chunk

    # 병렬 처리로 파일 읽기
    bag = db.from_sequence(file_names).map(process_file).compute()
    computed_bag = bag.compute()  # 이 부분에서 계산을 시작합니다.
    progress(computed_bag)  # 진척상황을 표시합니다.

    # 결과 데이터프레임 생성
    if computed_bag:
        final_df = dd.concat([chunk for sublist in computed_bag for chunk in sublist], axis=0)
        return final_df
    else:
        print("No dataframes to concatenate.")
        return None

In [7]:
def test_json_data_loading(directory, df):
    """데이터가 올바르게 DataFrame에 로드되었는지 테스트"""
    file_names = [f for f in os.listdir(directory) if f.startswith('output-') and f.endswith('.json')]
#     file_names = [f"output-{i}.json" for i in range(11)]
    
    total_attend_count = 0
    
    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            users = data.get("Users", {})
            for user_data in users.values():
                attend_data = user_data.get("Attend", {})
                total_attend_count += len(attend_data)
#     # 단일 파일로 테스트 용
#     with open(file_path, 'r', encoding='utf-8') as file:
#         data = json.load(file)
#     users = data.get("Users", {})
#     for user_data in users.values():
#         attend_data = user_data.get("Attend", {})
#         total_attend_count += len(attend_data)
    
    # Comparing the attend count in the files with the dataframe
    df_attend_count = df.shape[0]
    
    assert total_attend_count == df_attend_count, f"Attend count mismatch. Files: {total_attend_count}, DataFrame: {df_attend_count}"
    
    return "Test passed: Data loaded correctly into DataFrame."

In [8]:
def inspect_dataframe(df):
    """
    주어진 Dataframe의 구조와 몇몇 데이터를 보여준다
    
    Args:
    - df (DataFrame): 조사할 dataframe.
    """
    
    # 첫 5행 확인
    print("First 5 rows of the dataframe:")
    print(df.head())
    print("-" * 80)

    # 데이터프레임 정보 확인
    print("\nDataframe Info:")
    print(df.info())
    print("-" * 80)

    # 기술 통계 확인 (숫자 데이터에만 해당)
    print("\nDataframe Description:")
    print(df.describe())
    print("-" * 80)
    
    return

In [9]:
# attend 정보가 들어간 json 파일들을 df_attend에 로드
# df_attend = load_json_to_df(attend_directory)
df_attend = load_json_to_df_spark(attend_directory)

Py4JJavaError: An error occurred while calling o32.json.
: java.lang.OutOfMemoryError: Java heap space
	at scala.LowPriorityImplicits.wrapRefArray(Predef.scala:622)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.children(TreeNode.scala:1246)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.children$(TreeNode.scala:1246)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.children$lzycompute(Expression.scala:532)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.children(Expression.scala:532)
	at org.apache.spark.sql.catalyst.expressions.Expression._references$lzycompute(Expression.scala:125)
	at org.apache.spark.sql.catalyst.expressions.Expression._references(Expression.scala:124)
	at org.apache.spark.sql.catalyst.expressions.Expression.references(Expression.scala:127)
	at org.apache.spark.sql.catalyst.expressions.Expression.$anonfun$_references$1(Expression.scala:125)
	at org.apache.spark.sql.catalyst.expressions.Expression$$Lambda$2338/0x0000000801

In [11]:
gc.collect()

1499

In [10]:
# df_attend에 데이터가 올바르게 로드되었는지 test
test_result = test_json_data_loading(attend_directory, df_attend)
print(test_result)
# inspect_dataframe(df_attend)

NameError: name 'df_attend' is not defined

# 외부 디스크 데이터, df_external로 로드

In [ ]:
external_file_path = 'C:/Users/admin/Desktop/230517_d_95_extracted_ssd_d95_data_1.xlsx'  # 모든 외부 디스크
external_file_path1 = 'C:/Users/admin/Desktop/230517_d_80_gcs_extracted_hdd_d80_data_1.xlsx' 
external_file_path2 = 'C:/Users/admin/Desktop/230517_d_80_gcs_extracted_hdd_d80_data_2.xlsx'

df_ssd = pd.read_excel(file_path)   # 외부 디스크 통합
df_hdd1 = pd.read_excel(file_path)  
df_hdd2 = pd.read_excel(file_path1)
df_external = pd.concat([df_ssd, df_hdd1, df_hdd2], ignore_index=True)

num_rows = df.shape[0]
num_columns = df.shape[1]
print(f"File has \'{num_rows}\' rows and \'{num_columns}\' columns.")

# 유저 정보 데이터, df_users로 로드

In [ ]:
users_file_path = 'D:/DATA_PREPROCESS/Firestore_datas/users_data.csv'

df_users = pd.read_csv(users_file_path)

# 포인트 광고 / 암기플러스 광고 데이터, df_ad_point / df_ad_memo로 로드

In [ ]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# Google Cloud Console에 대한 서비스 키
creds_path = "C:/Users/admin/Desktop/voice_data_queries/samboss-reward-394470968e63.json"

# Spreadsheet ID는 해당 url 주소의 d와 edit 사이의 값
# spreadsheet_id = "1XMjIScbWqyiyzGMfOk5jvbAQ_ys8Srg8e60R6BkqMJ4"  # fs_point_20230801 시트
memo_sheet_id = "1fstoqfCCyIv38D4ZOxicoDdIaLhcfuXOf_9yn-ijn_U"   # fs_memo_20230801 시트

# Google Drive API와 Google Sheets API에 연결하고 인증
creds = Credentials.from_service_account_file(creds_path)
drive_service = build('drive', 'v3', credentials=creds)     # ( 사용하려는 API, API 버전, 인증 )
sheets_service = build('sheets', 'v4', credentials=creds)

# Spreadsheet에 대한 메타데이터
memo_sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=memo_sheet_id).execute()

# 시트의 이름을 지정 (예: 'Sheet1')
sheet_name = '시트1'

# # 범위를 지정 (예: 'Sheet1!A1:D10')
# range_name = f'{sheet_name}!A1:C10'
range_name = sheet_name  # 범위를 시트 이름만으로 지정 -> 전체 시트

# 시트의 내용 가져오기
memo_sheet = sheets_service.spreadsheets().values().get(spreadsheetId=memo_sheet_id, range=range_name).execute()

# 결과를 Pandas DataFrame으로 변환
df_ad_memo = pd.DataFrame(memo_sheet.get('values', []))

# 첫 번째 행을 열 이름으로 사용
df.columns = df.iloc[0]
df = df.iloc[1:]

# 결과 확인
print(df)